<a href="https://colab.research.google.com/github/juergenlandauer/FoundationModelsArchaeology/blob/main/Experiment_4_Hillforts_SAM_segmentation/SAM_hillforts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automatic site segmentation with SAM: English hillforts
Author: Juergen Landauer (juergen AT landauer-ai.de)

To start, first go to the "Input parameters" section below and review or (optionally) adjust parameters. Then run the entire Notebook by choosing Runtime->Run all in the menu above.

### Install SAM2 and dependencies

We first install Meta's SAM, Roboflow's Supervision and other libraries, then the SAM model weights. This could take up to 5 minutes.

In [ ]:
!pip install -Uq opencv-python supervision
!pip install -Uq 'git+https://github.com/facebookresearch/sam2.git'

### Caution!
You might get asked to restart this session here. This is mandatory.

# Input parameters

Review all parameters in this section and (optionally) adjust them on the right side. For example, you can upload your own input zip file by providing an URL.

In [ ]:
# feel free to replace this with your own imagery by providing a download URL (e.g. from Google Drive)
# Note that the ZIP file must contain only images in its root folder. No sub-directories!

INPUT_ZIP_URL = 'https://www.dropbox.com/scl/fi/u0htcr0kllnohu7hwsfox/DroneVideo.zip?rlkey=zxj171l892fgogi4mo8h5ej8e&st=33q8o1ka&dl=0' # @param {"allow-input":true}

## Setup

In [ ]:
# download SAM weights
!mkdir -p ../checkpoints/
#!wget -P ../checkpoints/ https://dl.fbaipublicfiles.com/segment_anything_2/092824/sam2.1_hiera_large.pt
!wget -P ../checkpoints/ https://dl.fbaipublicfiles.com/segment_anything_2/092824/sam2.1_hiera_small.pt

In [ ]:
from glob import glob
from pathlib import Path
from tqdm import tqdm
import albumentations as A
import os

In [ ]:
HOME = os.getcwd()
print("HOME:", HOME)

### Download SAM2 checkpoints

In [ ]:
#!mkdir -p {HOME}/checkpoints
#!cd {HOME}/checkpoints && \
#./download_ckpts.sh && \
#cd ..

### Imports

In [ ]:
import cv2
import torch
import base64

import numpy as np
import supervision as sv

**NOTE:** This code enables mixed-precision computing for faster deep learning. It uses bfloat16 for most calculations and, on newer NVIDIA GPUs, leverages TensorFloat-32 (TF32) for certain operations to further boost performance.

In [ ]:
torch.autocast(device_type="cuda", dtype=torch.bfloat16).__enter__()

if torch.cuda.get_device_properties(0).major >= 8:
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True

## Load model





In [ ]:
from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor
from sam2.automatic_mask_generator import SAM2AutomaticMaskGenerator

In [ ]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
CHECKPOINT = f"{HOME}/../checkpoints/sam2.1_hiera_small.pt"
CONFIG = "configs/sam2.1/sam2.1_hiera_s.yaml"

sam2_model = build_sam2(CONFIG, CHECKPOINT, device=DEVICE, apply_postprocessing=False)

## Automated mask generation

Since SAM 2 can efficiently process prompts, masks for the entire image can be generated by sampling a large number of prompts over an image.

The class `SAM2AutomaticMaskGenerator` implements this capability. It works by sampling single-point input prompts in a grid over the image, from each of which SAM can predict multiple masks. Then, masks are filtered for quality and deduplicated using non-maximal suppression. Additional options allow for further improvement of mask quality and quantity, such as running prediction on multiple crops of the image or postprocessing masks to remove small disconnected regions and holes.

In [ ]:
mask_generator = SAM2AutomaticMaskGenerator(
    sam2_model,
    points_per_side=64,
    points_per_batch=128,
    pred_iou_thresh=0.7,
    stability_score_thresh=0.92,
    stability_score_offset=0.7,
    crop_n_layers=1,
    box_nms_thresh=0.7,
)


**NOTE:** OpenCV loads images in BGR format by default, so we convert to RGB for compatibility with the mask generator.

In [ ]:
IMAGE_PATH = f"{HOME}/../../../06_Hillforts 2021/Data/DEM_forts_EN_768_V8_hillshade"

In [ ]:
def generate (image):
    image_rgb = image#cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

    sam2_result = mask_generator.generate(image_rgb)
    return sam2_result

### Output format

`SAM2AutomaticMaskGenerator` returns a `list` of masks, where each mask is a `dict` containing various information about the mask:

* `segmentation` - `[np.ndarray]` - the mask with `(W, H)` shape, and `bool` type
* `area` - `[int]` - the area of the mask in pixels
* `bbox` - `[List[int]]` - the boundary box of the mask in `xywh` format
* `predicted_iou` - `[float]` - the model's own prediction for the quality of the mask
* `point_coords` - `[List[List[float]]]` - the sampled input point that generated this mask
* `stability_score` - `[float]` - an additional measure of mask quality
* `crop_box` - `List[int]` - the crop of the image used to generate this mask in `xywh` format

### Results visualisation

In [ ]:
mask_annotator = sv.MaskAnnotator(color_lookup=sv.ColorLookup.INDEX)

def annotate(image_bgr, sam2_result, filename):
    detections = sv.Detections.from_sam(sam_result=sam2_result)
    annotated_image = mask_annotator.annotate(scene=image_bgr.copy(), detections=detections)

    sv.plot_images_grid(
        images=[image_bgr, annotated_image],
        grid_size=(1, 2),
        titles=['source image', 'segmented image']
    )

    if filename is not None:
        cv2.imwrite(filename, annotated_image)


In [ ]:
def load_rgb(image_path):
    image = cv2.imread(image_path)
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [ ]:
def preprocessSnippetNONE(image: np.ndarray): return image

In [ ]:
myfilter = preprocessSnippetNONE

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

def read_img(f):
    image = cv2.imread(f, cv2.IMREAD_UNCHANGED)
    #im = []
    #for s in [1.0, 2.0, 3.0]:
    #    image = myfilter(image, s).astype(np.uint8)
    #    im.append(image)
    #    print (image.min(), image.max(), image.shape)

    image = np.stack([image]*3, axis=2).astype(np.uint8)
    #image = np.stack(im, axis=2)
    #print (im)
    # minmax
    #norm = cv2.normalize(image, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    #image = norm

    image = myfilter(image).astype(np.uint8)

    #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    #image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    ##image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

    # colorize with colormap
    # see https://docs.opencv.org/4.x/d3/d50/group__imgproc__colormap.html
    #colorized = cv2.applyColorMap(image, cv2.COLORMAP_VIRIDIS)
    #colorized = cv2.applyColorMap(image, cv2.COLORMAP_SUMMER)
    #colorized = cv2.applyColorMap(image, cv2.COLORMAP_RAINBOW)
    #image = colorized

    # convert to HSV
    #img_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    #image = img_hsv

    return image

## process folder

In [ ]:
OUTPATH = Path("./outpathSAMhillfortsPointPrompts")
os.makedirs(OUTPATH, exist_ok=True)

files = sorted(glob(IMAGE_PATH+"/*tif"))
#files = files[:22]
#files

In [ ]:
sz = 768
default_box = [
    {'x': sz//2, 'y': sz//2, 'width': 0, 'height': 0, 'label': ''},
]

input_point = np.array([
    [sz//2, sz//2]
])
input_label = np.ones(input_point.shape[0])
input_label

In [ ]:
predictor = SAM2ImagePredictor(sam2_model)

In [ ]:
for f in tqdm(files):
    p = Path(f)
    image_bgr = read_img(p)
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

    predictor.set_image(image_rgb)
    masks, scores, logits = predictor.predict(
        point_coords=input_point,
        point_labels=input_label,
        multimask_output=True,
    )

    for s in range(len(scores)):
        if scores[s] == max(scores): # got it
            mask = masks[s].astype(np.uint8)
            break

    redImg = np.zeros_like(image_bgr)
    redImg[:,:] = (0, 255, 255)
    redMask = cv2.bitwise_and(redImg, redImg, mask=mask)

    overlay = cv2.addWeighted(redMask, .3, image_bgr, 1-.3, 0)

    sv.plot_images_grid(
        images=[image_bgr, overlay],
        titles=["orig", f"score: {scores[s]:.2f}"],
        grid_size=(1, 2),
        size=(12, 6)
    )
    # save mask and overlay
    cv2.imwrite(OUTPATH/("orig_"+p.name), image_bgr)
    cv2.imwrite(OUTPATH/("SAM_"+p.name), overlay)



## Export results for download
We now open a file download dialog for the output.zip. Simply store the output in your local computer. Done :-)

In [ ]:
!zip -r output.zip output
colabfiles.download('output.zip')